In [1]:
%matplotlib inline
import os
from pathlib import Path
from logging import getLogger
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

from src.models.utils import callbacks
from src.preprocessing.computer_vision import scaling, reshaping, resize, augmentation
from src.projects.fashion_mnist.architecture import CnnFCNNetwork

In [2]:
_log = getLogger()
model_save_fp = Path(os.getcwd()) / Path('trained_models') / Path('fashion_mnist')

Loading data from Keras

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

Ingesting data with the Tensorflow Dataset API and splitting between training and validation

In [4]:
full_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
training_dataset = full_dataset.take(50000)
validation_dataset = full_dataset.skip(50000)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-12-01 18:42:11.947103: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-01 18:42:11.947235: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Preprocessing data by scaling, rotating and reshaping the input images

In [5]:
def preprocess(dataset):
    dataset = dataset.map(lambda x, y: (scaling(x), y), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(lambda x, y: (reshaping(x, 28), y), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(lambda x, y: (augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

In [6]:
training_dataset = preprocess(training_dataset)
validation_dataset = preprocess(validation_dataset)

Batching and shuffling

In [7]:
batch_size = 64
training_dataset = training_dataset.shuffle(10000, reshuffle_each_iteration=True)
training_dataset = training_dataset.repeat(10)
training_dataset = training_dataset.batch(batch_size)
training_dataset = training_dataset.prefetch(tf.data.AUTOTUNE)

validation_dataset = validation_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

Model loading

In [9]:
name = 'CNNFCNNetwork'
artifacts_path = model_save_fp / Path(name)
model = CnnFCNNetwork(name=name, filepath=artifacts_path)
model = model.connect_layers()
epochs = 50
model.fit(
    x=training_dataset, 
    validation_data=validation_dataset, 
    epochs=epochs, 
    callbacks=callbacks()
)
model.save(artifacts_path)
_log.info(f"Saved {model.name} in folder {artifacts_path}")

Model: "CNNFCNNetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 1)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 64)    1664      
                                                                 
 batch_normalization (BatchN  (None, None, None, 64)   256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, None, None, 64)    0         
                                                                 
 dropout (Dropout)           (None, None, None, 64)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 128)   73856     
                                                     

2022-12-01 18:42:43.187482: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-01 18:42:43.187919: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7813/7813 [==============================] - ETA: 0s - loss: 0.5415 - accuracy: 0.8156

2022-12-01 18:44:36.170328: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7813/7813 [==============================] - 114s 14ms/step - loss: 0.5415 - accuracy: 0.8156 - val_loss: 0.5041 - val_accuracy: 0.8089 - lr: 0.0010
Epoch 2/50
7813/7813 [==============================] - 106s 13ms/step - loss: 0.3586 - accuracy: 0.8706 - val_loss: 0.3665 - val_accuracy: 0.8651 - lr: 0.0010
Epoch 3/50
7813/7813 [==============================] - 106s 13ms/step - loss: 0.3141 - accuracy: 0.8859 - val_loss: 0.3187 - val_accuracy: 0.8799 - lr: 0.0010
Epoch 4/50
7813/7813 [==============================] - 103s 13ms/step - loss: 0.2858 - accuracy: 0.8957 - val_loss: 0.2928 - val_accuracy: 0.8915 - lr: 0.0010
Epoch 5/50
7813/7813 [==============================] - 103s 13ms/step - loss: 0.2650 - accuracy: 0.9029 - val_loss: 0.3527 - val_accuracy: 0.8640 - lr: 0.0010
Epoch 6/50
7813/7813 [==============================] - 103s 13ms/step - loss: 0.2502 - accuracy: 0.9081 - val_loss: 0.3071 - val_accuracy: 0.8880 - lr: 0.0010
Epoch 7/50
7813/7813 [=============================

INFO:tensorflow:Assets written to: /Users/gianrond/git/deeplearning/trained_models/fashion_mnist/CNNFCNNetwork/assets


INFO:tensorflow:Assets written to: /Users/gianrond/git/deeplearning/trained_models/fashion_mnist/CNNFCNNetwork/assets


Performance evaluation against test data

In [ ]:
test_dataset = preprocess(test_dataset)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

Classification report and confusion matrix

In [ ]:
model_paths = {
    'CNNFCNNetwork' : model_save_fp / Path('CNNFCNNetwork'),
}
loaded_models = {}
for model_name, model_path in model_paths.items():
    loaded_models[model_name] = tf.keras.models.load_model(model_path)

In [10]:
accuracies = {}
for model_name, model in loaded_models.items():
    print(f'*** Model: {model_name} ***')
    accuracies[model_name] = model.evaluate(test_dataset)
    y_pred = model.predict(test_dataset)
    y_pred_argmax = np.argmax(y_pred, axis=1)

    cm = confusion_matrix(y_test, y_pred_argmax)
    print(cm)
    cr = classification_report(y_test, y_pred_argmax)
    print(cr)
print(accuracies)

NameError: name 'loaded_models' is not defined

In [ ]:
# Checking performance
score = model.evaluate(test_dataset)
y_pred = model.predict(test_dataset)
y_pred_argmax = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_test, y_pred_argmax)
print(cm)
cr = classification_report(y_test, y_pred_argmax)
print(cr)